In [21]:
import pandas as pd
from sentence_transformers import SentenceTransformer, util
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [13]:
df = pd.read_json("results/task2/task2_data.json", lines=True)
df

,title,code,iteration,ref_embeddings,gen_embeddings,model,prompt
0,Accountants and Auditors,13-2011.00,0,"[0.057781610600000005, 0.0170972049, 0.0063463...","[0.0417524837, 0.0190687012, -0.00704117400000...",deepseek-r1 7b,benchmark
1,Actors,27-2011.00,0,"[0.0500539839, -0.0298149604, 0.0063925134, -0...","[0.002087527, -0.0311016198, -0.0193263888, -0...",deepseek-r1 7b,benchmark
2,Actuaries,15-2011.00,0,"[0.0549289249, 0.0150179062, -0.016457418, -0....","[-0.040955919800000004, 0.0421547741, -0.02783...",deepseek-r1 7b,benchmark
3,Acupuncturists,29-1291.00,0,"[0.0539231524, -0.0455680303, 0.0001075003, -0...","[0.0418119878, -0.0549953133, -0.0264714062000...",deepseek-r1 7b,benchmark
4,Acute Care Nurses,29-1141.01,0,"[0.0543873385, -0.061012145100000005, -0.00616...","[0.0588279627, -0.0555830337, -0.0179473888000...",deepseek-r1 7b,benchmark
...,...,...,...,...,...,...,...
70315,Information Security Analysts,15-1212.00,2,"[0.057768281500000004, 0.0274809953, -0.012417...","[0.0343777537, 0.044208828400000004, -2.05231e...",mistral 7b,persona
70316,Information Technology Project Managers,15-1299.09,2,"[0.0582552738, -0.025281094, -0.02123377100000...","[0.0510931164, 0.0046637557, -0.02471387390000...",mistral 7b,persona
70317,"Inspectors, Testers, Sorters, Samplers, and We...",51-9061.00,2,"[-0.014055796, -0.0299870949, -0.0098509695, -...","[0.013496476200000001, -0.028076298500000003, ...",mistral 7b,persona
70318,Instructional Coordinators,25-9031.00,2,"[0.07710634920000001, -0.0424890332, -0.007097...","[0.0882729292, -0.039919771300000004, -0.03364...",mistral 7b,persona


In [16]:
#calculate the cosine similarity between the two columns
df['similarity'] = df.apply(lambda x: util.pytorch_cos_sim(x['ref_embeddings'], x['gen_embeddings']), axis=1)
df

,title,code,iteration,ref_embeddings,gen_embeddings,model,prompt,similarity
0,Accountants and Auditors,13-2011.00,0,"[0.057781610600000005, 0.0170972049, 0.0063463...","[0.0417524837, 0.0190687012, -0.00704117400000...",deepseek-r1 7b,benchmark,[[tensor(0.8067)]]
1,Actors,27-2011.00,0,"[0.0500539839, -0.0298149604, 0.0063925134, -0...","[0.002087527, -0.0311016198, -0.0193263888, -0...",deepseek-r1 7b,benchmark,[[tensor(0.6538)]]
2,Actuaries,15-2011.00,0,"[0.0549289249, 0.0150179062, -0.016457418, -0....","[-0.040955919800000004, 0.0421547741, -0.02783...",deepseek-r1 7b,benchmark,[[tensor(0.5388)]]
3,Acupuncturists,29-1291.00,0,"[0.0539231524, -0.0455680303, 0.0001075003, -0...","[0.0418119878, -0.0549953133, -0.0264714062000...",deepseek-r1 7b,benchmark,[[tensor(0.8742)]]
4,Acute Care Nurses,29-1141.01,0,"[0.0543873385, -0.061012145100000005, -0.00616...","[0.0588279627, -0.0555830337, -0.0179473888000...",deepseek-r1 7b,benchmark,[[tensor(0.8911)]]
...,...,...,...,...,...,...,...,...
70315,Information Security Analysts,15-1212.00,2,"[0.057768281500000004, 0.0274809953, -0.012417...","[0.0343777537, 0.044208828400000004, -2.05231e...",mistral 7b,persona,[[tensor(0.8263)]]
70316,Information Technology Project Managers,15-1299.09,2,"[0.0582552738, -0.025281094, -0.02123377100000...","[0.0510931164, 0.0046637557, -0.02471387390000...",mistral 7b,persona,[[tensor(0.9363)]]
70317,"Inspectors, Testers, Sorters, Samplers, and We...",51-9061.00,2,"[-0.014055796, -0.0299870949, -0.0098509695, -...","[0.013496476200000001, -0.028076298500000003, ...",mistral 7b,persona,[[tensor(0.7401)]]
70318,Instructional Coordinators,25-9031.00,2,"[0.07710634920000001, -0.0424890332, -0.007097...","[0.0882729292, -0.039919771300000004, -0.03364...",mistral 7b,persona,[[tensor(0.8673)]]


In [20]:
import pandas as pd
import numpy as np

def standardize_embeddings_array(emb_array: np.ndarray) -> np.ndarray:
    """
    Standardize a 2D array of embeddings (mean=0, std=1 per column).
    """
    mean = np.mean(emb_array, axis=0)
    std = np.std(emb_array, axis=0)
    std[std == 0] = 1e-8  # Prevent divide-by-zero
    return (emb_array - mean) / std

# --- Step 1: Expand lists to 2D arrays ---
ref_array = np.vstack(df["ref_embeddings"].to_numpy())
gen_array = np.vstack(df["gen_embeddings"].to_numpy())

# --- Step 2: Standardize ---
ref_std = standardize_embeddings_array(ref_array)
gen_std = standardize_embeddings_array(gen_array)

# --- Step 3: Store back in DataFrame ---
df["ref_std"] = list(ref_std)
df["gen_std"] = list(gen_std)


In [24]:
df_std = df.copy()
df_std = df_std.drop(columns=["ref_embeddings", "gen_embeddings", "similarity"])
df_std["cos_sim_std"] = [
    cosine_similarity([r], [g])[0][0]
    for r, g in zip(df_std["ref_std"], df_std["gen_std"])
]
df_std

,title,code,iteration,model,prompt,ref_std,gen_std,cos_sim_std
0,Accountants and Auditors,13-2011.00,0,deepseek-r1 7b,benchmark,"[0.6570865635195471, 0.9003317561814614, 0.719...","[0.1683640143129726, 0.5934520015282692, 0.218...",0.636852
1,Actors,27-2011.00,0,deepseek-r1 7b,benchmark,"[0.4637579537666792, -0.5628888425809871, 0.72...","[-0.9761356023458323, -0.8989827402906713, -0....",0.598449
2,Actuaries,15-2011.00,0,deepseek-r1 7b,benchmark,"[0.5857184990602627, 0.8354770928044124, -0.87...","[-2.2181187648352965, 1.2802017882821697, -1.1...",0.408810
3,Acupuncturists,29-1291.00,0,deepseek-r1 7b,benchmark,"[0.5605562329853931, -1.0542372214933695, 0.28...","[0.17008095604959683, -1.6097571100574781, -1....",0.866381
4,Acute Care Nurses,29-1141.01,0,deepseek-r1 7b,benchmark,"[0.5721691714545359, -1.5359490921882828, -0.1...","[0.6610628799029473, -1.6272402420357592, -0.4...",0.773098
...,...,...,...,...,...,...,...,...
70315,Information Security Analysts,15-1212.00,2,mistral 7b,persona,"[0.6567530980880121, 1.2242088552100105, -0.59...","[-0.044427738412228066, 1.341304486805916, 0.6...",0.698968
70316,Information Technology Project Managers,15-1299.09,2,mistral 7b,persona,"[0.668936598659973, -0.4214746344306559, -1.21...","[0.43788026830400983, 0.16494286126966076, -0....",0.858831
70317,"Inspectors, Testers, Sorters, Samplers, and We...",51-9061.00,2,mistral 7b,persona,"[-1.1401309374538748, -0.5682578283130884, -0....","[-0.6469397813434667, -0.8089874099182616, -1....",0.522595
70318,Instructional Coordinators,25-9031.00,2,mistral 7b,persona,"[1.1405499853972603, -0.9582013222455074, -0.2...","[1.5106731070529331, -1.1612994924043922, -1.4...",0.779383
